# 1. Problema de negócio

Atualmente, a previsão de vendas não é suficientemente precisa para direcionar a produção de forma eficiente. Isso pode levar a desajustes entre a oferta e a demanda, resultando em excesso ou falta de estoque. Por conta disso é necessário criar medidas para poder analisar os produtos que devem ser fabricados e quando deve fabricá-los.


## 1.1. Resultado Esperado

Desenvolver e implementar um modelo avançado de previsão de vendas que permita uma estimativa precisa da demanda. Este modelo deve fornecer informações detalhadas sobre quais produtos devem ser fabricados e os momentos ideais para sua produção. A partir dessa previsão precisa, a indústria conseguirá alinhar a produção com a demanda real, otimizando a gestão de estoque e evitando tanto excessos quanto faltas. 

# 2 - Entendimento dos dados e Formulação de Hipóteses

## 2.1. Variáveis

| **Feature**              | **Tipo Variável** | **Descrição**                                                |
|--------------------------|--------------------|--------------------------------------------------------------|
| COD_CICLO                | Ordinal            | Indicação de período definido por marketing                  |
| FLG_DATA_COMEMORATIVA    | FLAG               | Indicação se o ciclo pertence a alguma data comemorativa nacional |
| COD_MATERIAL             | Categórica         | Código referente ao material - SKU                           |
| COD_CANAL                | Categórica         | Em qual canal de vendas foi efetuada a venda do período       |
| DES_CATEGORIA_MATERIAL   | Categórica         | Qual a categoria do material                                 |
| DES_MARCA_MATERIAL       | Categórica         | Qual a marca do material                                     |
| COD_REGIAO               | Categórica         | Região a qual foi efetuada a venda                           |
| QT_VENDA_BRUTO           | Continua           | Quantidade de itens vendidos                                |
| QT_DEVOLUCAO             | Continua           | Quantidade de itens devolvidos                              |
| VL_RECEITA_BRUTA         | Continua           | Valor da receita bruta                                      |
| VL_RECEITA_LIQUIDA       | Continua           | Valor da receita líquida                                    |
| FLG_CAMPANHA_MKT_A       | FLAG               | Campanha de Marketing - TIPO A                              |
| FLG_CAMPANHA_MKT_B       | FLAG               | Campanha de Marketing - TIPO B                              |
| FLG_CAMPANHA_MKT_C       | FLAG               | Campanha de Marketing - TIPO C                              |
| FLG_CAMPANHA_MKT_D       | FLAG               | Campanha de Marketing - TIPO D                              |
| FLG_CAMPANHA_MKT_E       | FLAG               | Campanha de Marketing - TIPO E                              |
| PCT_DESCONTO             | Continua           | Percentual de desconto feito por Marketing                   |
| VL_PRECO                 | Continua           | Valor do preço do Produto                                    |


# 3. Coleta e preparação dos dados

## 3.1. Instalação das bibliotecas

In [1]:
"""
# análise estatística dos dados: jupyter-summarytools

# Analise de correlações: seaborn e heatmapz

# Remoção de acentos: unidecode
"""

import os
import sys
import subprocess

libraries = ["pandas", "scikit-learn"]


for lib in libraries:
    try:
        __import__(lib)
        print(f"{lib} já esta instalado.")
    except ImportError:
        print(f"{lib} não encontrado. Iniciando a instalação")
        subprocess.check_call([sys.executable, "-m", "pip", "install", lib])
        print(f"{lib} instalado com sucesso.")
    

pandas já esta instalado.
scikit-learn não encontrado. Iniciando a instalação
scikit-learn instalado com sucesso.


## 3.2. Importando as bibliotecas e setando funções 

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from unidecode import unidecode
from datetime import datetime, timedelta

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from category_encoders import BinaryEncoder, CountEncoder
from sklearn.preprocessing import RobustScaler

In [3]:
def week_to_date(data, day=1):
    """
    Converte o ano e número da semana para o primeiro dia da semana
    """
    
    year = int(str(data)[:4])
    week = int(str(data)[4:])
    
    # Calcular o primeiro dia do ano
    start_date = datetime(year, 1, 1)
    
    # Ajustar para a primeira segunda-feira do ano
    start_date += timedelta(days=(7 - start_date.weekday()))
    
    # Calcular o início da semana desejada
    start_of_week = start_date + timedelta(weeks=week - 1)
    
    # Ajustar para o dia da semana (1 para segunda-feira, ..., 7 para domingo)
    target_date = start_of_week + timedelta(days=day - 1)
    
    return target_date

## 3.3. Lendo o dataset

In [4]:
# Obtendo o dataset
df = pd.read_csv("dataset.csv", delimiter = ';')

df.head()

,COD_CICLO,FLG_DATA,COD_MATERIAL,COD_CANAL,DES_CATEGORIA_MATERIAL,DES_MARCA_MATERIAL,COD_REGIAO,QT_VENDA_BRUTO,QT_DEVOLUCAO,VL_RECEITA_BRUTA,VL_RECEITA_LIQUIDA,FLG_CAMPANHA_MKT_A,FLG_CAMPANHA_MKT_B,FLG_CAMPANHA_MKT_C,FLG_CAMPANHA_MKT_D,FLG_CAMPANHA_MKT_E,PCT_DESCONTO,VL_PRECO
0,201917,1,431148,anon_S0,anon_S2,anon_S3,anon_S1,"11934,000000",414.0,"431869,080000","431869,080000",0,0,0,0,0,NaN,"455,400000"
1,202005,0,177816,anon_S0,anon_S2,anon_S4,anon_S1,"540,000000",252.0,"27743,400000","27743,400000",0,0,0,0,0,NaN,"773,400000"
2,201901,0,171786,anon_S0,anon_S5,anon_S6,anon_S1,"54012,000000",1410.0,"962860,200000","962860,200000",0,1,0,0,0,"35,000000","341,400000"
3,201813,0,177774,anon_S7,anon_S2,anon_S8,anon_S1,"438,000000",NaN,"7608,600000","7608,600000",0,0,0,0,0,NaN,"450,900000"
4,202006,1,446592,anon_S0,anon_S5,anon_S9,anon_S1,"2760,000000",240.0,"83339,400000","83339,400000",0,0,0,0,0,NaN,"431,400000"


In [5]:
# Obtendo o dicionário com as informações das colunas
df_dictionario = pd.read_csv("dicionario de dados.csv", delimiter = ',', header = 1)

# Ajuste nos nomes das colunas para que fique minusculo, sem acento e sem espaço
df_dictionario.columns = [unidecode(x).lower().replace(" ", "_") for x in df_dictionario.columns]

df_dictionario.head()

,feature,tipo_variavel,descricao
0,COD_CICLO,Ordinal,Indicação de período definido por marketing
1,FLG_DATA_COMEMORATIVA,FLAG,Indicação se o ciclo pertence a alguma data co...
2,COD_MATERIAL,Categórica,Código referente ao material - SKU
3,COD_CANAL,Categórica,Em qual canal de vendas foi efetuada a venda d...
4,DES_CATEGORIA_MATERIAL,Categórica,Qual a categoria do material


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173923 entries, 0 to 173922
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   COD_CICLO               173923 non-null  int64  
 1   FLG_DATA                173923 non-null  int64  
 2   COD_MATERIAL            173923 non-null  int64  
 3   COD_CANAL               173923 non-null  object 
 4   DES_CATEGORIA_MATERIAL  173923 non-null  object 
 5   DES_MARCA_MATERIAL      173923 non-null  object 
 6   COD_REGIAO              173923 non-null  object 
 7   QT_VENDA_BRUTO          173923 non-null  object 
 8   QT_DEVOLUCAO            87164 non-null   float64
 9   VL_RECEITA_BRUTA        173923 non-null  object 
 10  VL_RECEITA_LIQUIDA      173923 non-null  object 
 11  FLG_CAMPANHA_MKT_A      173923 non-null  int64  
 12  FLG_CAMPANHA_MKT_B      173923 non-null  int64  
 13  FLG_CAMPANHA_MKT_C      173923 non-null  int64  
 14  FLG_CAMPANHA_MKT_D  

# 4. Tratamento dos dados

## 4.1. Ajustando o tipo de valores

In [7]:
# Criando as listas com os tipos de cada coluna
tipos_variaveis = df_dictionario.groupby('tipo_variavel')['feature'].apply(list).to_dict()

# Removendo o FLG_DATA_COMEMORATIVA pois não tem no dataframe
tipos_variaveis['FLAG'].remove('FLG_DATA_COMEMORATIVA')

lista_categoricas = tipos_variaveis.get('Categórica', [])
lista_ordinal = tipos_variaveis.get('Ordinal', [])
lista_flag = tipos_variaveis.get('FLAG', [])
lista_continua = tipos_variaveis.get('Continua', [])

# Ajustando de objeto para float
for i in df.columns:
    if i in lista_continua and df[i].dtype != 'float64':
        df[i] = df[i].apply(lambda x: float(x.replace(',', '.')) if pd.notna(x) else x)

## 4.2. Preenchendo valores vazios

In [8]:
"""
As colunas que tem valores nulos são QT_DEVOLUCAO e PCT_DESCONTO, nesse caso vou considerar como sendo 0, ou seja,
não teve devolução e não teve pacote de desconto
"""
df = df.fillna(0.0)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173923 entries, 0 to 173922
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   COD_CICLO               173923 non-null  int64  
 1   FLG_DATA                173923 non-null  int64  
 2   COD_MATERIAL            173923 non-null  int64  
 3   COD_CANAL               173923 non-null  object 
 4   DES_CATEGORIA_MATERIAL  173923 non-null  object 
 5   DES_MARCA_MATERIAL      173923 non-null  object 
 6   COD_REGIAO              173923 non-null  object 
 7   QT_VENDA_BRUTO          173923 non-null  float64
 8   QT_DEVOLUCAO            173923 non-null  float64
 9   VL_RECEITA_BRUTA        173923 non-null  float64
 10  VL_RECEITA_LIQUIDA      173923 non-null  float64
 11  FLG_CAMPANHA_MKT_A      173923 non-null  int64  
 12  FLG_CAMPANHA_MKT_B      173923 non-null  int64  
 13  FLG_CAMPANHA_MKT_C      173923 non-null  int64  
 14  FLG_CAMPANHA_MKT_D  

## 4.3. Alterar os dados da coluna COD_CICLO de string para datetime e criando as features de data

In [9]:
"""
Os valores do COD_CICLO inicia com o ano seguido de um número com dois valores, como é maior que 12 
então não é o mês, sendo assim considerei como sendo a semana do ano
"""

df['COD_CICLO'].unique()

array([201917, 202005, 201901, 201813, 202006, 202017, 201814, 202007,
       201804, 201818, 201802, 201806, 202010, 201913, 201812, 201809,
       202004, 201805, 201910, 201807, 202011, 202015, 201912, 202003,
       201903, 202009, 201904, 201911, 201801, 201914, 202012, 202014,
       201909, 202013, 201916, 201907, 201803, 202008, 201902, 201905,
       202002, 201810, 201815, 201816, 201808, 201817, 202016, 202101,
       201908, 201906, 201811, 202001, 201915], dtype=int64)

In [10]:
# Ajustando os valores da coluna para que seja datetime
df['data'] = df['COD_CICLO'].apply(week_to_date)

In [11]:
# Criar features a partir da data
df['ano'] = df['data'].dt.year
df['mes'] = df['data'].dt.month
df['dia'] = df['data'].dt.day
df['dia_da_semana'] = df['data'].dt.dayofweek
df['semana_do_ano'] = df['data'].dt.isocalendar().week
df['dia_do_ano'] = df['data'].dt.dayofyear

# 5. Escolha do modelo


o RandomForestRegressor pode ser eficaz porque permite capturar padrões complexos e não lineares nas vendas, ajudando a fazer previsões mais precisas e robustas com base em várias características que afetam as vendas.

In [12]:
# Dividir dados
X = df.drop(['data', 'VL_RECEITA_LIQUIDA'], axis=1)
y = df['VL_RECEITA_LIQUIDA']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Definindo as métricas de ajuste das colunas

"""
- O ColumnTransformer permite aplicar transformações diferentes para tipos diferentes de colunas nos dados
- O RobustScaler normaliza os dados numéricos que possuem outliers

"""
t = [('num', RobustScaler(), ['ano', 'mes', 'dia', 'dia_da_semana', 'semana_do_ano', 'dia_do_ano'])]

preprocessor = ColumnTransformer(transformers = t)


In [14]:
# Pipeline de pré-processamento e modelagem
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Treinar o modelo
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', RobustScaler(),
                                                  ['ano', 'mes', 'dia',
                                                   'dia_da_semana',
                                                   'semana_do_ano',
                                                   'dia_do_ano'])])),
                ('model', RandomForestRegressor(random_state=42))])

In [15]:
"""
MSE é útil quando queremos penalizar grandes erros mais severamente. Pode ser mais sensível a outliers, pois o erro é elevado ao quadrado.
RMSE é útil para uma interpretação mais direta do erro médio. Ele proporciona uma visão mais intuitiva de quanto, em média, o modelo está errando.

O resultado 6213853254385.732 de Mean Squared Error indica que o resultado do modelo não foi adequado e que deve ser melhorado o processo

"""

# Prever e avaliar
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")


Mean Squared Error: 6213853254385.732
Root Mean Squared Error: 2492760.1678432147


In [16]:
# Para acessar a importância das features, primeiro obtenha o regressor do pipeline
regressor = pipeline.named_steps['model']

# Para obter as importâncias das features, precisamos acessar as importâncias
# Aplicar o mesmo pré-processamento às features do conjunto de teste
X_test_transformed = pipeline.named_steps['preprocessor'].transform(X_test)

# Obter a importância das features diretamente do modelo
importances = regressor.feature_importances_

In [17]:
"""
Nesse caso a feature que tem mais importância é o ano, o que indica que a tendência é aumentar 
"""

# Os nomes das features após o pré-processamento são mantidos em um formato que pode não corresponder aos originais
# Portanto, é necessário mapear as importâncias para os nomes originais
features = ['ano', 'mes', 'dia', 'dia_da_semana', 'semana_do_ano', 'dia_do_ano']

importance_df = pd.DataFrame({'Feature': features, 'Importance': importances})

importance_df = importance_df.sort_values(by='Importance', ascending=False)

print("\nImportância das Features:")

print(importance_df)


Importância das Features:
         Feature  Importance
5     dia_do_ano    0.509262
4  semana_do_ano    0.259910
2            dia    0.161542
0            ano    0.064835
1            mes    0.004451
3  dia_da_semana    0.000000
